# Harmony Service Chaining: PI 20.4 Demo

In PI 20.4, Harmony add service chaining capabilities to support requests that require functionality beyond that of a single service.
This notebook provides a basic workflow to demonstrate service chaining. For more a general introduction and tutorial, see [Harmony API Introduction](./Harmony%20Api%20Introduction.ipynb).  Useful helpers for making the calls found in this notebook can be found under the [docs/notebook-helpers](./notebook-helpers) folder.

## Prerequisites

1. Install Python 3. This notebook is tested to work in 3.8 but should work in most recent 3.x versions.
2. Install Jupyter: pip install jupyterlab
3. Setup your ~/.netrc for Earthdata Login as described in Harmony API Introduction
4. Run the following cell to install Python dependencies, import necessary modules, and set notebook defaults

In [ ]:
%load_ext autoreload
%autoreload
%matplotlib inline

import sys
# Install dependencies into the Jupyter Kernel
!{sys.executable} -m pip install -q -r notebook_helpers/requirements.txt

# Import libraries used throughout the notebook
from notebook_helpers import get, post, show, get_data_urls, show_async, show_async_condensed, show_shape, print_async_status, check_bbox_subset, check_stac

## Chained Services - PODAAC L1 Subsetter -> Harmony NetCDF to Zarr

This request asks for variable subsetting of L1 data with output in the Zarr format. This requires chaining two services together, the PODAAC L1 Subsetter and the Harmony NetCDF to Zarr service.

In [ ]:
podaac_collection = 'C1234208438-POCLOUD'
coverages_root = 'https://harmony.sit.earthdata.nasa.gov/{collection}/ogc-api-coverages/1.0.0/collections/{variable}/coverage/rangeset'


### Variable and spatial subsetting with reformtatting output to Zarr and spatial constraints

In [ ]:
response = get(
    coverages_root.format(
        collection=podaac_collection, 
        variable='mean%5Fsea%5fsurface'),
    params={
        'format': 'application/x-zarr',
        'subset': [
            'lon(-160:-160)', 
            'lat(-80:80)'
            ],
        'maxResults': '2'})

show_async(response)